### import library

In [50]:
from requests import get
from bs4 import BeautifulSoup
import unicodecsv as csv
import pandas as pd
import numpy as np
import folium
from googlegeocoder import GoogleGeocoder
import unicodecsv as csv

#### make soup

In [25]:
url = 'https://blog.benchsci.com/startups-using-artificial-intelligence-in-drug-discovery'
response = get(url)

soup = BeautifulSoup(response.text, 'html.parser')

#### create CSV file

In [31]:
csv_file = open('start_up_141.csv', 'ab')
writer = csv.writer(csv_file, dialect='excel', delimiter=',', encoding='utf-8')
#writer.writerow(['Search_Term', 'Result', 'Title', 'URL', 'Abstract'])
# writer.writerow(['comapnay','city', 'state', 'full_address', 'zip_code'])
writer.writerow(['compnay','website'])

17

In [27]:
containter = soup.find_all('h3')

In [28]:
len(containter)

143

In [32]:
# last 2 does not conain any link so -2
for link in containter[:-2]:
    company = link.text
    website = link.a["href"]
    writer.writerow([company, website])
csv_file.close()
    

In [33]:
import pandas as pd

In [44]:
df = pd.read_csv('start_up_141.csv')

In [45]:
df.head()

,compnay,website
0,BioSymetrics,http://biosymetrics.com
1,Biorelate,http://biorelate.com
2,Causaly,http://causaly.com
3,Data2Discovery,http://d2discovery.com
4,Data4Cure,http://data4cure.com


In [36]:
df.shape

(141, 2)

In [41]:
links = list(df['website'])

In [42]:
len(links)

141

In [ ]:
# all link to txt file

In [43]:
# with open('start_up.txt', 'w') as f:
#     for item in links:
#         f.write("%s\n" % item)
  
    

In [56]:
df_add = pd.read_csv('start_up_141_address.csv', encoding = 'latin-1')

In [57]:
df_add.head()

,name,web,address
0,A2A Pharmaceuticals,http://a2apharma.com,"101 6th Ave, New York, NY 10013"
1,Acellera,http://acellera.com,"Devonshire House 582 Honeypot Lane Stanmore, ..."
2,Acurastem,http://acurastem.com,"605 E. Huntington Dr., Suite 103 Monrovia, CA ..."
3,Aetion,http://aetion.com,"1441 Broadway20th Floor New York, NY 10018"
4,AI Therapeutics,https://www.ai-therapeutics.com,"530 Old Whitfield Street Guilford, CT 06437"


In [58]:
df_add.shape

(138, 3)

#### get latitude and longitude

In [59]:
geocoder = GoogleGeocoder("your key")

In [60]:
csv_file = open('start_up_ai.csv', 'ab')
writer = csv.writer(csv_file, dialect='excel', delimiter=',', encoding='utf-8')
#writer.writerow(['Search_Term', 'Result', 'Title', 'URL', 'Abstract'])
writer.writerow(['lat', 'lng'])

9

In [61]:
df_address = df_add['address']

In [62]:
# address that does not work save into list
no_lat =[]
for k in df_address:
    try:
        search = geocoder.get(k)
        lat = search[0].geometry.location.lat
        lng = search[0].geometry.location.lng
        writer.writerow([lat, lng])
        
    except:
        no_lat.append(k)
        pass
csv_file.close()  



In [63]:
no_lat

[]

In [64]:
start_up_lat = pd.read_csv('start_up_ai.csv')

In [65]:
start_up_lat.head()

,lat,lng
0,40.723421,-74.005453
1,51.603353,-0.297531
2,34.141117,-117.991952
3,40.754925,-73.987141
4,41.272590,-72.665423


In [66]:
start_up_lat.shape

(138, 2)

#### merge two dataframe

In [67]:
df_map = pd.concat([df_add, start_up_lat], axis=1)

In [70]:
df_map.columns

Index(['name', 'web', 'address', 'lat', 'lng'], dtype='object')

In [69]:
df_map.to_csv('df_start_lat.csv',index=False, )

#### create map

In [72]:
lat = list(df_map["lat"])
lon =list(df_map["lng"])
name =list(df_map["name"])
# address_full = list(df_map["address"])
web = list(df_map['web'])


map = folium.Map(location=[35.199,-101.91], zoom_start=5)

fg=folium.FeatureGroup(name="My Map")

for lt,ln,nm, web  in zip(lat, lon,name, web):
    fg.add_child(folium.Marker(location=[lt,ln],
       popup= folium.Popup(str(nm)+ " "+ "|"+ str(web)),
                               icon = folium.Icon()))

    map.add_child(fg)

map.save("start_up_AI.html")